---
## **Sujet** : Prédiction de diagnostic de performance énergétique (DPE) d'habitats dans les départements de l'Ardèche (07) et de la Haute-Corse (2A)
#### **Auteurs** : Mélodie FLEURY, Matéo PETITET 
#### **Matière** : UE Prog supervisée par Vincent GUIGUE - 3A IODAA AgroParisTech
#### **Date de rendue** : 18 novembre 2024
---

##### Import des libairies nécessaires

In [1]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import seaborn as sns
import matplotlib as plt
import time
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import *
from sklearn import svm,linear_model, naive_bayes
from sklearn.ensemble import *
import sklearn.metrics as metrics
import xgboost as xgb
import optuna

c:\Users\Melo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Les fonctions utilisées dans ce notebook

In [2]:
def load_file_dpe(file_dpe):
    """
    Charge un fichier csv et le transforme en DataFrame

    Input : 
    file_dpe (str) : Fichier csv à charger

    Output :
    dpe_data (DataFrame) : Fichier transformé en DataFrame
    """
    #Permet d'afficher l'ensemble des colonnes
    pd.set_option('display.max_columns', None)
    #Lecture du jeu de données (format csv)
    dpe_data = pd.read_csv(
        file_dpe,
        sep = ",")
    print("Dimension du jeu de données : \n", np.shape(dpe_data)) #(49275, 131)
    #Affichage des 5 premières lignes du jeu de données
    dpe_data.head(5)
    
    return dpe_data

def select_classe_valid(df):
    """
    Sélectionne les classes de DPE valides

    Input :
    df (DataFrame) : DataFrame avec toutes les classes de DPE

    Output :
    df (DataFrame) : DataFrame avec sélection de classes de DPE
    """
    #Dataframe ne comportant pas les habitats ayant pour classe N
    valeur_correct = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
    df = df[(df['classe_consommation_energie'].isin(valeur_correct))].reset_index()
    
    return df

def type_conso_energie(df):
    """
    Création et remplissage de l'attribut "type_consommation_energie"

    Input :
    df (DataFrame) : DataFrame sans l'attribut "type_consommation_energie"

    Output :
    df (DataFrame) : DataFrame avec l'attribut "type_consommation_energie"
    """
    #Création attribut "type_consommation_energie"
    df["type_consommation_energie"] = ''
    #Conversion en type numérique de l'attribut "type_consommation_energie"
    df["type_consommation_energie"] = df["type_consommation_energie"].apply(pd.to_numeric)

    #Association des classes de consommations d'énergie aux types
    dict_classe_type = {
        'A' : 1,
        'B' : 2,
        'C' : 3,
        'D' : 4,
        'E' : 5,
        'F' : 6,
        'G' : 7
    }

    for i in range(len(df['classe_consommation_energie'])):
        df.loc[i,'type_consommation_energie'] = dict_classe_type[df.loc[i,'classe_consommation_energie']]

    return df

def supp_attrib_id(df):
    """
    Supprime les attributs de type "identifiant"

    Input :
    df (DataFrame) : DataFrame avec les attributs de type "identifiant"

    Ouput :
    df (DataFrame) : DataFrame sans les attributs de type "identifiant"
    """
    for colonne in df.columns:
        if "_id" in colonne or colonne == "id":
            print(colonne)
            df = df.drop(columns=colonne)

    return df

def supp_attrib_null(df):
    """
    Supprime les attributs ayant plus de 50% de leurs valeurs nulles

    Input :
    df (DataFrame) : DataFrame avec les attributs (valeur nulles > 50%)

    Ouput :
    df (DataFrame) : DataFrame sans les attributs (valeurs nulles > 50%)
    """
    #Fixation du seuil de suppression en nombre de lignes
    seuil_f = 0.5 * len(df)
    #Suppression des colonnes avec plus de 50% de valeurs manquantes
    colonnes_a_supprimer_f = df.columns[df.isnull().sum() > seuil_f]
    df = df.drop(columns=colonnes_a_supprimer_f)
    print(f"Colonnes supprimées : {list(colonnes_a_supprimer_f)}")
    
    return df

def moy_val_null(df):
    """
    Supprime les valeurs nulles restantes en les remplaçant par
    la moyenne de toutes les valeurs de l'attribut (réservés
    aux attributs de type numérique)

    Input :
    df (DataFrame) : DataFrame avec les valeurs nulles

    Output :
    df (DataFrame) : DataFrame sans valeurs nulles (seulement
    pour les attributs de type numérique)
    """
	#Imputation des valeurs manquantes des colonnes avec valeurs restantes
    colonnes_a_imputer_f = df.columns[df.isnull().sum()>0]
    print(f"Colonnes amputées : {list(colonnes_a_imputer_f)}")
    imputer = SimpleImputer(strategy='mean') #on utilise la moyenne
    df = pd.DataFrame(df)
    if len(colonnes_a_imputer_f) > 0 :
        for colonne in colonnes_a_imputer_f:
             if is_numeric_dtype(df[colonne]) is True: #filtre sur les attributs numériques
                print(colonne)
                X_imput=imputer.fit_transform(df[[colonne]])
                df[[colonne]] = pd.DataFrame(X_imput, columns=[colonne], index=df.index)
                
    return df

def corr_attrib_target(df):
    """
    Fait la corrélation entre le descripteur target et tous les autres 
    descripteurs restants. Puis, sélectionne les descripteurs ayant 
    une valeur de corrélation supérieure à 0.1 qui seront utilisés
    ensuite pour l'apprentissage des modèles

    Input : 
    df (DataFrame) : DataFrame sans valeurs nulles

    Output :
    name_columns (List) : Liste de tous les descripteurs sélectionnés
    pour l'apprentissage
    """
    #corrélation entre tous les attributs et la classe "type_consommation_energie"
    corr_target = df.corrwith(
        df["type_consommation_energie"], 
        method='pearson',
        drop = True)
    print(corr_target)
    corr_target_df = pd.DataFrame(corr_target).reset_index()
    corr_target_df.columns = ['attribut', 'correlation_value']
    #transformation en valeur absolue des valeurs de corrélation
    corr_target_df["correlation_value"] = corr_target_df["correlation_value"].abs()
    #sélection des taux de corrélation supérieur à 0.1 et tri en ordre croissant
    #Les descripteurs sélectionnés pour l'apprentissage
    corr_target_df[corr_target_df["correlation_value"] > 0.1].sort_values("correlation_value")
    name_columns = corr_target_df[corr_target_df["correlation_value"] > 0.1].sort_values("correlation_value").attribut.tolist()
    print("name_columns", name_columns)
    
    return name_columns

def create_df_final(df, name_columns):
    """
    Création du DataFrame final pour l'apprentissage des classifieurs

    Input : 
    df (DataFrame) : DataFrame sans valeurs nulles
    name_columns (List) : Liste de tous les descripteurs sélectionnés
    pour l'apprentissage

    Output : 
    dpe_df_final (DataFrame) : DataFrame final
    """
    #Création du dataframe final avec les descripteurs finaux pour cette première méthode de sélection de variables
    dpe_df_final = pd.DataFrame()
    for name in name_columns :
        print(name)
        dpe_df_final[df.loc[:, name].name] = df.loc[:, name]
    
    return dpe_df_final


##### Chargement du fichier de données (.csv) des habitats ayant fait un DPE dans le département de l'Ardèche

In [3]:
dpe_data = load_file_dpe("dep_7.csv")

Dimension du jeu de données : 
 (49275, 131)


C:\Users\Melo\AppData\Local\Temp\ipykernel_2184\967949434.py:14: DtypeWarning: Columns (27,34,35,36,37,43,76,92,93,94,96,97,100) have mixed types. Specify dtype option on import or set low_memory=False.
  dpe_data = pd.read_csv(


<span style="color:orange"> <u> Description des données : </u> Nous avons ici une base de données de 49 275 habitats situés dans le département de l'Ardèche dont on connaît leur diagnostique de performance énergétique, et plus de 131 descripteurs dont "classe_consommation_energie" qui est l'attribut que l'on souhaite prédire ici. Pour plus d'informations sur la source de ces données, c'est ici : [lien](https://data.ademe.fr/datasets/dpe-france). </span>

<span style="color:orange"> Quelques remarques peuvent être faites sur les données, il y a un grand nombre de descripteurs dont certains ont une majorité nulle et sont déséquilibrés au niveau de leur classe. Ces attributs seront soit éliminés soit non sélectionnés lors de l'étape de réduction de dimensionnalité. Ensuite, il y a beaucoup d'attributs qui sont de type texte (string), ce sont donc des attributs difficiles à manipuler car il est nécessaire de les transformer en attribut numérique pour le classifieur ensuite. Par exemple, le descripteur cible "classe_consommation_energie".</span>

##### Analyse et pré-traitements du descripteur target "classe_consommation_energie"
<span style="color:orange"> Le descripteur à prédire "classe_consommation_energie" prend la forme d'une lettre et va de A à G avec A : le logement a une très bonne performance énergétique et climatique, et G : le logement a une très mauvaise performance énergétique et climatique. Seulement, on remarque la présence de la classe "N" qui n'appartient à aucune classe d'énergie et qui est majoritaire dans ce jeu de données (13 839 habitats). La dictionnaire des données associé à ce jeu de données [lien](https://koumoul.com/s/data-fair/api/v1/datasets/dpe-france/metadata-attachments/ADEME%20-%20DPE%20-%20Dictionnaire%20de%20donn%C3%A9es%20-%202020-06-08.pdf) ne mentionne pas cette classe. En analysant le jeu de données, il semblerait que ce soit les logements ayant fait le DPE mais jugés non admissibles. Ainsi, on va supprimer les lignes portant cette classe ou toute autre classe qui ne font pas partie d'une classe correcte (lettre de A à G). </span>

In [19]:
print(
    "Les différentes étiquettes de consommation d'énergie du jeu de données sont : \n",
      dpe_data["classe_consommation_energie"].unique()
    )

print(
    "Effectif de chaque classe : \n", 
    dpe_data['classe_consommation_energie'].value_counts()
    )

print(
    "Les habitats portant la classe N : \n", 
    dpe_data[(dpe_data['classe_consommation_energie'] =='N')].head(2)
    )

dpe_data_valid = select_classe_valid(dpe_data)

print(
    "Effectif de chaque classe avec la suppression de la classe N \n", 
    dpe_data_valid['classe_consommation_energie'].value_counts()
)

Les différentes étiquettes de consommation d'énergie du jeu de données sont : 
 ['D' 'F' 'E' 'C' 'A' 'G' 'N' 'B']
Effectif de chaque classe : 
 N    13839
D     9854
E     7901
C     5774
A     4617
F     3477
B     2274
G     1539
Name: classe_consommation_energie, dtype: int64
Les habitats portant la classe N : 
       id     numero_dpe  usr_diagnostiqueur_id  usr_logiciel_id  \
20  4575  1307V1000045F                   7334                3   
26  5563  1307V2000003A                   1415                3   

    tr001_modele_dpe_id  nom_methode_dpe   version_methode_dpe  \
20                    1      Méthode 3CL  3CL-DPE, version 1.3   
26                    2  Méthode Facture                   NaN   

   nom_methode_etude_thermique version_methode_etude_thermique  \
20                         NaN                             NaN   
26                         NaN                             NaN   

   date_visite_diagnostiqueur date_etablissement_dpe  \
20                 2013-05-

<span style="color:orange"> Maintenant, ce descripteur target va être transformé en un descripteur de type numérique appelé "type_consommation_energie". Cela permet de faciliter l'apprentissage du classifieur ensuite. Concrètement, un habitat, ayant pour classe de consommation d'énergie la lettre A, portera le chiffre 1, un habitat, ayant pour classe de consommation d'énergie la lettre B, portera le chiffre 2, etc.</span>

In [20]:
dpe_data_valid = type_conso_energie(dpe_data_valid)

#### Sélection des descripteurs
<span style="color:orange"> L'étape ici est de sélectionner les attributs les plus pertinents pour la classe target afin d'avoir de bonnes performances avec le classifieur. En effet, selon le principe de "fléau de dimensionnalité", le classifieur va être bon avec les données d'apprentissage mais médiocre avec les données tests. </span>

##### Suppression des descripteurs de type identifiant
<span style="color:orange"> Tout d'abord, on va supprimer les attributs de type identifiant car ils n'apportent aucune information interprétable sur le diagnostic de performance énergétique des habitats. </span>

In [21]:
dpe_data_valid = supp_attrib_id(dpe_data_valid)

id
usr_diagnostiqueur_id
usr_logiciel_id
tr001_modele_dpe_id
tr002_type_batiment_id
tr012_categorie_erp_id
tr013_type_erp_id
tv016_departement_id
geo_id
tr001_modele_dpe_type_id
tr013_type_erp_categorie_id
tv017_zone_hiver_id
tv018_zone_ete_id


<span style="color:orange"> La liste ci-dessus montre alors les attributs type "identifiant" supprimés. </span>

##### Gestion des valeurs manquantes
<span style="color:orange">  Dans un premier temps, on va alors s'intéresser aux valeurs nulles et supprimer les attributs qui ont plus de 50% de valeurs nulles.</span>

In [22]:
dpe_data_valid = supp_attrib_null(dpe_data_valid)

Colonnes supprimées : ['nom_methode_etude_thermique', 'version_methode_etude_thermique', 'commentaires_ameliorations_recommandations', 'explication_personnalisee', 'secteur_activite', 'arrondissement', 'type_voie', 'numero_rue', 'batiment', 'escalier', 'etage', 'porte', 'numero_lot', 'quote_part', 'nom_centre_commercial', 'surface_commerciale_contractuelle', 'partie_batiment', 'type_vitrage_verriere', 'geo_l5', 'tr012_categorie_erp_code', 'tr012_categorie_erp_categorie', 'tr012_categorie_erp_groupe', 'tr012_categorie_erp_est_efface', 'tr013_type_erp_code', 'tr013_type_erp_type', 'tr013_type_erp_est_efface', 'tr013_type_erp_categorie', 'tv016_departement_c4']


In [ ]:
np.shape(dpe_data_valid)

(35436, 92)

<span style="color:orange"> Ce deuxième filtre va permettre de moyenner les valeurs manquantes des attributs numériques seulement.</span>

In [ ]:
dpe_data_valid = moy_val_null(dpe_data_valid)

Colonnes amputées : ['version_methode_dpe', 'surface_habitable', 'surface_thermique_lot', 'commune', 'nom_rue', 'code_postal', 'code_insee_commune', 'code_insee_commune_actualise', 'portee_dpe_batiment', 'shon', 'surface_utile', 'surface_thermique_parties_communes', 'en_souterrain', 'en_surface', 'nombre_niveaux', 'nombre_circulations_verticales', 'nombre_boutiques', 'presence_verriere', 'surface_verriere', 'nombre_entrees_avec_sas', 'nombre_entrees_sans_sas', 'surface_baies_orientees_nord', 'surface_baies_orientees_est_ouest', 'surface_baies_orientees_sud', 'surface_planchers_hauts_deperditifs', 'surface_planchers_bas_deperditifs', 'surface_parois_verticales_opaques_deperditives', 'organisme_certificateur', 'adresse_organisme_certificateur', 'dpe_vierge', 'longitude', 'latitude', 'geo_type', 'geo_adresse', 'geo_l4']
surface_habitable
surface_thermique_lot
portee_dpe_batiment
shon
surface_utile
surface_thermique_parties_communes
en_souterrain
en_surface
nombre_niveaux
nombre_circulatio

##### Méthode de corrélation entre les descripteurs et le descripteur target
<span style="color:orange"> Cette méthode de sélection de variables est plutôt naîve mais efficace. On va étudier la corrélation entre les variables de la table "dpe_data_noN" et le descripteur target "type_consommation_energie". Une fois que les valeurs de corrélation entre chaque descripteur et le descripteur target sont calculées, on sélectionne les variables qui ont une valeur de corrélation supérieure à 0,1. Ces descripteurs seront utilisés ensuite pour l'apprentissage du classifieur.</span>

In [25]:
name_columns = corr_attrib_target(dpe_data_valid)

index                                -0.048874
consommation_energie                  0.389406
estimation_ges                        0.189500
annee_construction                    0.288418
surface_habitable                    -0.096276
                                        ...   
tv018_zone_ete_sclim_inf_150               NaN
tv018_zone_ete_sclim_sup_150               NaN
tv018_zone_ete_rclim_autres_etages         NaN
tv018_zone_ete_rclim_dernier_etage         NaN
type_consommation_energie             1.000000
Length: 61, dtype: float64
name_columns ['surface_baies_orientees_sud', 'surface_parois_verticales_opaques_deperditives', 'surface_planchers_hauts_deperditifs', 'en_surface', 'estimation_ges', 'surface_planchers_bas_deperditifs', 'dpe_vierge', 'annee_construction', 'consommation_energie', 'tr001_modele_dpe_type_ordre', 'type_consommation_energie']


In [26]:
dpe_df_final = create_df_final(df = dpe_data_valid, name_columns = name_columns)
print(dpe_df_final)

surface_baies_orientees_sud
surface_parois_verticales_opaques_deperditives
surface_planchers_hauts_deperditifs
en_surface
estimation_ges
surface_planchers_bas_deperditifs
dpe_vierge
annee_construction
consommation_energie
tr001_modele_dpe_type_ordre
type_consommation_energie
       surface_baies_orientees_sud  \
0                         0.000000   
1                         0.000000   
2                         0.000000   
3                         0.000000   
4                         0.000000   
...                            ...   
35431                     0.000000   
35432                     0.000000   
35433                     0.000000   
35434                     0.000000   
35435                     3.477383   

       surface_parois_verticales_opaques_deperditives  \
0                                            0.000000   
1                                            0.000000   
2                                            0.000000   
3                                      

<span style="color:orange"> Finalement, avec la méthode de corrélation, 10 attributs sont sélectionnés dans l'apprentissage du classifieur. Ce sont des attributs en lien avec la consommation d'énergie et de gaz à effet de serre de l'habitat, avec l'année de sa construction ou bien avec sa superficie et son isolation thermique. Tous ces attributs sont enregistrés dans un dataframe final appelé : "dpe_df_final". </span>

##### Sélection des modèles
<span style="color:orange"> Ensuite, il faut choisir le ou les modèles qui vont prédire les diagnostics de performance des habitats. Ici, on va tester le modèle de bayésien naïf, le modèle svm et des approches ensemblistes telles que random forest et XGBoosting. Ce sont tous des modèles de classification supervisée.</span>

In [ ]:
#Séparation des attributs entre ceux qui vont aider à prédire (Xn) le descripteur target (y)
Xn = dpe_df_final.iloc[:, :-1]
print("Xn : \n", Xn)
y = dpe_df_final.iloc[:,-1]
print("y : \n", y)
#Séparation des données d'apprentissage et des données de tests
X_train, X_test, y_train, y_test = train_test_split(Xn, y, test_size=0.33, random_state=0)

Xn : 
        surface_baies_orientees_sud  \
0                         0.000000   
1                         0.000000   
2                         0.000000   
3                         0.000000   
4                         0.000000   
...                            ...   
35431                     0.000000   
35432                     0.000000   
35433                     0.000000   
35434                     0.000000   
35435                     3.477383   

       surface_parois_verticales_opaques_deperditives  \
0                                            0.000000   
1                                            0.000000   
2                                            0.000000   
3                                            0.000000   
4                                            0.000000   
...                                               ...   
35431                                       89.500000   
35432                                       89.500000   
35433                  

In [ ]:
# Comparaison des modèles
mod_bn = naive_bayes.GaussianNB() #modèle bayésien naïf
mod_svm = svm.SVC() #modèle SVM
mod_SVM1 = svm.SVC(kernel="linear")
mod_SVM2 = svm.SVC(gamma = 10)
mod_rf = RandomForestClassifier() #modèle random forest

# apprentissage sur les données d'apprentissage
mod_bn.fit(X_train, y_train)
print("mod_bn")
mod_svm.fit(X_train, y_train)
print("mod_svm")
mod_SVM1.fit(X_train, y_train)
print("mod_SVM1")
mod_SVM2.fit(X_train, y_train)
print("mod_SVM2")
mod_rf.fit(X_train, y_train)
print(mod_rf)

mod_bn
mod_svm
mod_SVM1
mod_SVM2
RandomForestClassifier()


In [ ]:
# prédiction sur les données tests
yhat_bn = mod_bn.predict(X_test)
yhat_svm = mod_svm.predict(X_test)
yhat1 = mod_SVM1.predict(X_test)
yhat2 = mod_SVM2.predict(X_test)
yhat_rf = mod_rf.predict(X_test)

# évaluation de la performance des modèles grâce à la précision
print("précision modèle bayésien naïf ", metrics.accuracy_score(y_test, yhat_bn))
print("précision modèle svm ", metrics.accuracy_score(y_test, yhat_svm))
print("précision modèle svm1 ", metrics.accuracy_score(y_test, yhat1))
print("précision modèle svm2 ", metrics.accuracy_score(y_test, yhat2))
print("précision modèle random forest ", metrics.accuracy_score(y_test, yhat_rf))


précision modèle bayésien naïf  0.9020010261672653
précision modèle svm  0.9017444843509492
précision modèle svm1  0.9970925260817514
précision modèle svm2  0.4435608004104669
précision modèle random forest  0.9988883187959637


In [ ]:
mod_xgb = GradientBoostingClassifier() #modèle xgboost
mod_xgb.fit(X_train, y_train)
yhat_xgb = mod_xgb.predict(X_test)
print("précision modèle xgboost ", metrics.accuracy_score(y_test, yhat_xgb))

perf modèle xgboost  0.9992303745510518


<span style="color:orange"> En comparant la performance des 6 classifieurs, on se rend donc compte que ce sont les modèles svm1, random forest et xgboost qui sont les meilleurs avec 99 % de bonnes prédictions. </span>

##### Amélioration de la performance
<span style="color:orange"> En complément, il existe des outils intéressants pour améliorer la performance des classifieurs comme optuna qui selon un nombre d'itération choisie, sélectionne les meilleures hyperparamètres du modèle. Dans notre cas, cela n'est pas nécessaire car nous atteignons une performance proche de 1 sans amélioration de performance. Nous avons fait le choix de laisser le code de cet outil qui est à retrouver ci-dessous. </span>

In [ ]:
# Application d'optuna
#Define an objective function to be minimized.
def objective(trial):
    dpe_data = Xn
    dpe_target = y

    #invoke suggest methods of a trial object to generate hyperparameters (RandomForest or XGBoost)
    classifier = trial.suggest_categorical("classifier", ["RandomForest", "XGBoost"])

    #Hyperparamètres qui vont être testés dans Optuna
    if classifier == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 2, 20) #2 et 20 inclus
        max_depth = int(trial.suggest_float("max_depth", 1, 32, log=True))

        clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    
    else:
        n_estimators = trial.suggest_int("n_estimators", 2, 20)
        max_depth = int(trial.suggest_float("max_depth", 1, 32, log=True))
        clf = GradientBoostingClassifier(n_estimators= n_estimators, max_depth= max_depth)
    
    return cross_val_score(
        clf, dpe_data, dpe_target, n_jobs=-1, cv=3
    ).mean()

#create a new study
study = optuna.create_study(direction="maximize")
#lancement de l'optimisation
study.optimize(objective, n_trials=100)
#meilleure combinaison de paramètres
trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Le meilleur modèle selon optuna est le classifieur XGBoost avec 9 estimateurs et une profondeur de l'arbre égale à 4.50 environ.

##### Test sur un autre département : Cas de la Haute-Corse (2A)
<span style="color:orange"> Cette dernière partie se veut pratique. On va, en effet, tester les meilleurs modèles sur des données de la même source mais provenant d'un autre département (la Haute-Corse dans notre cas). </span>

<span style="color:orange"> Tout d'abord, certaines fonctions vont être réutilisées pour mettre en forme l'attribut "type_consommation_energie" et supprimer les attributs et les valeurs nulles. Enfin, les mêmes descripteurs sélectionnés initialement pour les habitats du département de l'Ardèche sont choisis pour le département de la Haute-Corse. </span>

In [ ]:
# Chargement fichier des habitats du département de la Haute-Corse évalués pour leur performance énergétique
dpe_data_2A = load_file_dpe("dep_2A.csv")
# Selection des classes de consommation énergétique valides
dpe_data_2A_valid = select_classe_valid(dpe_data_2A)
# Création de l'attribut "type_consommation_energie"
dpe_data_2A_valid = type_conso_energie(dpe_data_2A_valid)
# Suppression des attributs ayant plus de 50% de valeurs nulles
dpe_data_2A_valid = supp_attrib_null(dpe_data_2A_valid)
# Suppression des valeurs nulles en faisant la moyenne des valeurs de l'attribut correspondant
dpe_data_2A_valid = moy_val_null(dpe_data_2A_valid)
# Creation du dataframe final en sélectionnant les mêmes attributs que ceux du département de l'Ardèche
dpe_df_final_2A = create_df_final(df = dpe_data_2A_valid, name_columns = name_columns)
print(dpe_df_final_2A)

C:\Users\Melo\AppData\Local\Temp\ipykernel_13280\3050137707.py:5: DtypeWarning: Columns (27,35,43,92,93,94,96,97,100) have mixed types. Specify dtype option on import or set low_memory=False.
  dpe_data = pd.read_csv(


Dimension du jeu de données : 
 (19241, 131)
Colonnes supprimées : ['nom_methode_etude_thermique', 'version_methode_etude_thermique', 'commentaires_ameliorations_recommandations', 'explication_personnalisee', 'secteur_activite', 'tr012_categorie_erp_id', 'tr013_type_erp_id', 'arrondissement', 'type_voie', 'numero_rue', 'batiment', 'escalier', 'etage', 'porte', 'quote_part', 'nom_centre_commercial', 'surface_commerciale_contractuelle', 'partie_batiment', 'type_vitrage_verriere', 'longitude', 'latitude', 'geo_type', 'geo_adresse', 'geo_id', 'geo_l4', 'geo_l5', 'tr012_categorie_erp_code', 'tr012_categorie_erp_categorie', 'tr012_categorie_erp_groupe', 'tr012_categorie_erp_est_efface', 'tr013_type_erp_code', 'tr013_type_erp_type', 'tr013_type_erp_categorie_id', 'tr013_type_erp_est_efface', 'tr013_type_erp_categorie']
Colonnes amputées : ['version_methode_dpe', 'commune', 'nom_rue', 'code_postal', 'code_insee_commune', 'code_insee_commune_actualise', 'numero_lot', 'portee_dpe_batiment', 'sho

In [ ]:
# Vérification s'il y a bien que des classes valides
dpe_data_2A_valid["classe_consommation_energie"].unique()

array(['C', 'D', 'F', 'E', 'G', 'B', 'A'], dtype=object)

In [ ]:
# Tous les attributs utiles pour le classifieur
Xn_2A = dpe_df_final_2A.iloc[:, :-1]
print("Xn_2A : \n", Xn_2A)
# L'attribut objectif
y_2A = dpe_df_final_2A.iloc[:,-1]
print("y_2A : \n", y_2A)

Xn_2A : 
        surface_baies_orientees_sud  \
0                              0.0   
1                              0.0   
2                              0.0   
3                              0.0   
4                              0.0   
...                            ...   
17532                          0.0   
17533                          0.0   
17534                          0.0   
17535                          0.3   
17536                          0.0   

       surface_parois_verticales_opaques_deperditives  \
0                                                0.00   
1                                                0.00   
2                                                0.00   
3                                                0.00   
4                                                0.00   
...                                               ...   
17532                                           42.50   
17533                                           42.50   
17534               

<span style="color:orange"> Sélection des modèles avec le maximum de précision et test sur de nouveaux exemples d'apprentissages appartenant au département de la Haute-Corse </span>

In [62]:
yhat_svm1_2A = mod_SVM1.predict(Xn_2A)
yhat_rf_2A = mod_rf.predict(Xn_2A)
yhat_xgb_2A = mod_xgb.predict(Xn_2A)

print("précision modèle svm1 dep 2A ", metrics.accuracy_score(y_2A, yhat_svm1_2A))
print("précision modèle random forest dep 2A ", metrics.accuracy_score(y_2A, yhat_rf_2A))
print("précision modèle xgboost dep 2A ", metrics.accuracy_score(y_2A, yhat_xgb_2A))

précision modèle svm1 dep 2A  0.9936705251753436
précision modèle random forest dep 2A  0.9940126589496493
précision modèle xgboost dep 2A  0.9997719108171295


<span style="color:orange"> Finalement, on se rend compte qu'après avoir appliqué les modèles avec le plus de performance, on obtient également de très bon taux de prédiction pour les habitats de la Haute-Corse. En effet, pour chacun des modèles soit svm avec une fonction noyau linéaire, les forêts aléatoires et le XGBoosting, la précision atteint 99 % de bonnes prédictions. Alors, ces modèles pourraient sûrement être appliqués à l'échelle de tous les départements français voire même permettre à partir de quelques attributs (10 choisis ici environ) de savoir directement la valeur de diagnostic de performance énergétique de n'importe quel habitat si ces attributs sont renseignés. </span>